# Gerando o Dataset De Entrada De Acidentes

### Importando bibliotecas

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

### Lendo a planilha de Acidentes Pre processada
(Já com os Locais de instalação associados a cada Acidente)

In [ ]:
path = "data/"
path = ajuste_path(path)
pathUtil = path + "util/"

TabelaAcidentesPadronizada = pd.read_csv(
    pathUtil + "acidentes/" + "BD_Acidentes_Padronizado.csv", sep='|'
)
TabelaAcidentesPadronizada.shape

## Filtrando para apenas Acidentes da Eletrosul

In [ ]:
TabelaAcidentesPadronizada = TabelaAcidentesPadronizada[
    TabelaAcidentesPadronizada['Empresa'] == 'cgt eletrosul']
TabelaAcidentesPadronizada

### Contando a Quantidade de Acidentes
#### Em cada Local de instalação por Ano e Mês

In [ ]:
def contagemAcidentes(TabelaAcidentes):
    # Supondo que cada linha na tabela é um acidente, podemos usar size() para contar
    TabelaAcidentes = (
        TabelaAcidentes.groupby(["Ano", "Mês", "Local de instalação"])
        .size()
        .reset_index(name="Quantidade de Acidentes")
    )
    return TabelaAcidentes

In [ ]:
DatasetEntradaAcidentes = TabelaAcidentesPadronizada.copy()
DatasetEntradaAcidentes = contagemAcidentes(DatasetEntradaAcidentes)
DatasetEntradaAcidentes

### Contando a Quantidade de Acidentes de ALTO POTENCIAL
#### Em cada Local de instalação por Ano e Mês

In [ ]:
# Acidentes de Alto Potencial são Acidentes com Potencial de 1 a 8
acidentesAltoPotencial = TabelaAcidentesPadronizada[TabelaAcidentesPadronizada["Potencial"] < 9]

acidentesAltoPotencialPorMesAnoLocal = (
    acidentesAltoPotencial.groupby(["Ano", "Mês", "Local de instalação"])
    .size()
    .reset_index(name="Quantidade de Acidentes de Alto Potencial")
)
acidentesAltoPotencialPorMesAnoLocal

### Adicionando Colunas no Dataset de Entrada de Acidentes:

#### Quantidade de Acidentes de Alto Potencial

In [ ]:
def quantidadeDeAcidentesDeAltoPotencial(linha):
    dataset_filtrado = acidentesAltoPotencialPorMesAnoLocal[
        (acidentesAltoPotencialPorMesAnoLocal["Ano"] == linha["Ano"]) &
        (acidentesAltoPotencialPorMesAnoLocal["Mês"] == linha["Mês"]) &
        (acidentesAltoPotencialPorMesAnoLocal["Local de instalação"]
         == linha["Local de instalação"])
    ]
    if len(dataset_filtrado) > 0:
        return dataset_filtrado["Quantidade de Acidentes de Alto Potencial"].values[0]
    else:
        return 0


DatasetEntradaAcidentes["Quantidade de Acidentes de Alto Potencial"] = DatasetEntradaAcidentes.apply(
    quantidadeDeAcidentesDeAltoPotencial, axis=1)

#### Binario Acidentes de Alto Potencial
(1 se houver acidente de alto potencial, 0 caso contrário)

In [ ]:
def BinarioAcidentesAltoPotencial(linha):
    count = acidentesAltoPotencialPorMesAnoLocal[
        (acidentesAltoPotencialPorMesAnoLocal["Ano"] == linha["Ano"]) &
        (acidentesAltoPotencialPorMesAnoLocal["Mês"] == linha["Mês"]) &
        (acidentesAltoPotencialPorMesAnoLocal["Local de instalação"]
         == linha["Local de instalação"])
    ].shape[0]

    return count


DatasetEntradaAcidentes["Binario Acidentes de Alto Potencial"] = DatasetEntradaAcidentes.apply(
    BinarioAcidentesAltoPotencial, axis=1)

### Versao Inicial do Dataset de Entrada de Acidentes:

In [ ]:
DatasetEntradaAcidentes

## Validação do Dataset de Entrada de Acidentes

In [ ]:
# Comparar Quantidade de Acidentes por ano e mes na tabela original com o novo dataset entrada acidentes

acidentesPorAnoMesOriginal = (
    TabelaAcidentesPadronizada.groupby(["Ano", "Mês"])
    .size()
    .reset_index(name="Quantidade de Acidentes Original")
)

acidentesPorAnoMes = (
    DatasetEntradaAcidentes.groupby(["Ano", "Mês"])["Quantidade de Acidentes"]
    .sum()
    .reset_index()
)

# Left outer join para comprarar os datasets
resultado_join = pd.merge(
    acidentesPorAnoMesOriginal,
    acidentesPorAnoMes[["Ano", "Mês", "Quantidade de Acidentes"]],
    on=["Ano", "Mês"],
    how="left",
)

# Preencher NaN com 0 na coluna 'Quantidade de Acidentes'
resultado_join["Quantidade de Acidentes"] = resultado_join[
    "Quantidade de Acidentes"
].fillna(0)

# Para garantir que a coluna 'Quantidade de Acidentes' seja do tipo inteiro
resultado_join["Quantidade de Acidentes"] = resultado_join[
    "Quantidade de Acidentes"
].astype(int)

print(resultado_join)

## Conclusão da Validação:

In [ ]:
# soma dos counts totais de acidentes de cada uma das tabelas
print(acidentesPorAnoMesOriginal["Quantidade de Acidentes Original"].sum())
print(acidentesPorAnoMes["Quantidade de Acidentes"].sum())

### Diferenças entre os Datasets são causadas pela ausência do Local de Instação em 10 Linhas de Acidentes
Esses Acidentes estão há uma distancia maior do que 50 km (valor arbitrario) de qualquer Local de Instalação.

In [ ]:
# usando tabela TabelaAcidentesPadronizada para contar Quantidade de Acidentes sem local de instalação
qtdAcidentesSemLocalInstalacao = TabelaAcidentesPadronizada["Local de instalação"].isna(
).sum()
print(qtdAcidentesSemLocalInstalacao)

### Juntando Ano e Mês em uma unica coluna:
#### Ano Mes

In [ ]:
# juntando ano e mes numa unica coluna
DatasetEntradaAcidentes["Ano Mes"] = DatasetEntradaAcidentes["Ano"].astype(
    str) + "-" + DatasetEntradaAcidentes["Mês"].astype(int).astype(str).str.zfill(2)

# dropando colunas ano e mes antigas
DatasetEntradaAcidentes.drop(columns=["Ano", "Mês"], inplace=True)
DatasetEntradaAcidentes = DatasetEntradaAcidentes[[
    "Ano Mes", "Local de instalação", "Quantidade de Acidentes", "Quantidade de Acidentes de Alto Potencial", "Binario Acidentes de Alto Potencial"]]

## Versao Final do Dataset de Entrada de Acidentes:

In [ ]:
DatasetEntradaAcidentes

### Transformando o Dataset de Entrada de Acidentes em CSV

In [ ]:
DatasetEntradaAcidentes.to_csv(
    pathUtil + "acidentes/" + "acidentes_entrada.csv", index=False
)